### Preamble

> _If you've reached this part of the tutorial, you're a **Star**_ 🌟 🙌
> 
> **Thank you!** I hope you're finding this useful, and interesting!
> 
> _Valerio_

# Ensemble learning to combine multiple ML models

In the last notebook of our tutorial, we will explore how to use [Ensemble learning](https://en.wikipedia.org/wiki/Ensemble_learning) as a potential strategy to combine multiple machine learning models, trained on different datasets.

## Step 1. Login to datasites as **External Researcher**

⚠️ First verify that the Datasites are already running. If needed, launch the following command in a new terminal session:

```bash
$ python launch_datasites.py
```

**Note**: In Jupyter Lab, you can open a new terminal session via `File >> New >> Terminal`

In [ ]:
import syft as sy

In [ ]:
from datasites import DATASITE_URLS

datasites = {}
for name, url in DATASITE_URLS.items():
    datasites[name] = sy.login(url=url, email="researcher@openmined.org", password="****")

## Step 2. Prepare Ensemble Classifier

In [ ]:
mock_data = datasites["Cleveland Clinic"].datasets["Heart Disease Dataset"].assets["Heart Study Data"].mock

In [ ]:
# load pre-trained models
from utils import load_models

models = load_models(datasites, root="./models")

In [ ]:
# Explore "Voting Classifier" as Ensemble model strategy  
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

y = mock_data["num"].map(lambda v: 0 if v == 0 else 1)

# assign weights based on performance calculated in Experiment 03
voting_model = VotingClassifier(estimators=[(name, model) for name, model in models.items()], 
                                weights=[2,1,0.2,0.5])

# workaround to by-pass re-fit: https://stackoverflow.com/a/54610569
voting_model.estimators_ = list(models.values())
voting_model.le_ = LabelEncoder().fit(y)
voting_model.classes_ = voting_model.le_.classes_

voting_model

In [ ]:
# DS/ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as mcc_score
from sklearn.metrics import confusion_matrix


# ML Data preparation - same strategy as in model training
def by_demographics(data):
    sex = data["sex"].map(lambda v: '0' if v == 0 else '1')
    target = data["num"].map(lambda v: '0' if v == 0 else '1')
    return (sex+target).values


# 1. get features and labels
X = mock_data.drop(columns=["age", "sex", "num"], axis=1)
y = mock_data["num"].map(lambda v: 0 if v == 0 else 1)
# 2. partition data
_, X_test, _, y_test = train_test_split(
    X, y, random_state=12345, stratify=by_demographics(mock_data)
)
y_pred = voting_model.predict(X_test)
print(mcc_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## Step 3. Submit Experiment to each datasite

_Another evaluation experiment, this time using an `EnsembleVotingClassifier` model on each datasite!_

> 🙋 **Note**: The code of the `evaluate` function we are going to create is **identical** to the one we used in our previous evaluation experiment (see [`03-Model-Evaluation-Experiment.ipynb`](./03-Model-Evaluation-Experiment.ipynb)).
>
> It has been replicated here for clarity - but a **better way** to handle this would've been to import the code from a module, and just change the input parameter of the `syft_function_single_use` decorator!

In [ ]:
from utils import serialize_and_upload

remote_models = {}
for name, datasite in datasites.items():
    print(f"Datasite: {name}")
    # 1. Get data asset from datasite
    data_asset = datasite.datasets["Heart Disease Dataset"].assets["Heart Study Data"]
    # 1.1 Upload and serialise the Voting classifier to each Datasite
    remote_voting_model = serialize_and_upload(model=voting_model, to=datasite)
    remote_models[name] = remote_voting_model
    
    @sy.syft_function_single_use(data=data_asset, model=remote_voting_model)
    def evaluate(data, model):
        # DS/ML libraries
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import matthews_corrcoef as mcc_score
        from sklearn.metrics import confusion_matrix
        import joblib  # to load serialised input model
        
        # ML Data preparation - same strategy as in model training
        def by_demographics(data):
            sex = data["sex"].map(lambda v: '0' if v == 0 else '1')
            target = data["num"].map(lambda v: '0' if v == 0 else '1')
            return (sex+target).values
        
        # 1. get features and labels
        X = data.drop(columns=["age", "sex", "num"], axis=1)
        y = data["num"].map(lambda v: 0 if v == 0 else 1)
        # 2. partition data
        _, X_test, _, y_test = train_test_split(
            X, y, random_state=12345, stratify=by_demographics(data)
        )
        # 3. Get trained model
        classifier = joblib.load(model)  # load model
        # 4. Evaluate Metrics (MCC and Confusion Matrix)
        y_pred = classifier.predict(X_test)
        return mcc_score(y_test, y_pred), confusion_matrix(y_test, y_pred)
    
    ensemble_ml_project = sy.Project(
        name="Evaluate Ensable Voting Classifier on Heart Study Data",
        description="""This time I would like to evaluate the performance of an
        Ensemble Voting Models, using pre-trained models on each datasite.""",
        members=[datasite],
    )
    ensemble_ml_project.create_code_request(evaluate, datasite)
    project = ensemble_ml_project.send()

In [ ]:
from utils import check_status_last_code_requests

check_status_last_code_requests(datasites)

## Step 4. Evaluate Models on all datasites

In [ ]:
mcc_scores, confusion_matrices = {}, {}
for name, datasite in datasites.items():
    print(f"Datasite: {name}")
    data_asset = datasite.datasets["Heart Disease Dataset"].assets["Heart Study Data"]
    remote_model = remote_models[name]
    results = datasite.code.evaluate(data=data_asset, model=remote_model).get_from(datasite)
    mcc_scores[name], confusion_matrices[name] = results

**Note**: 
MCC Classification report of the `RandomForest` classifiers trained independently on each Datasites

```
'Cleveland Clinic': 0.766,
'Hungarian Inst. of Cardiology': 0.46,
'Univ. Hospitals Zurich and Basel': 0.0,
'V.A. Medical Center': 0.258}
```

In [ ]:
for name, score in mcc_scores.items():
    print(f"{name}: {score:.3f}")

In [ ]:
from matplotlib import pyplot as plt
from utils import plot_all_confusion_matrices

plot_all_confusion_matrices(confusion_matrices)
plt.show()

## Conclusions 🎉

Using an Ensemble of all models, we have been able to _improve_ the results of our classifier on **all** the four datasets by _indirectly_ incorporating the knowledge gathered from each (non-public) dataset, independently!

And we did this by **never, ever** _seeing_ the true private data stored in each datasite!